## Taylor Series examples

Created by: Dimitris Giovanis, 23/3/2019


The first-order reliability methods (FORM), as stated by its name, utilizes a first-order Taylor series expansions for the performance function in a standard normal probability space to derive probability of failure estimates. Consider a model in standard normal space  with performance function $g(\textbf{U})$. The FORM approximates
the performance function by:

$g(\textbf{U})  \approx L(\textbf{U}) = g(\textbf{u}^\star) + \nabla g(\textbf{u}^\star)(\textbf{U}-\textbf{u}^\star)^T$


where $\textbf{u}^\star$ is the point around which the series is expanded and its typically called the design point (needs to be found) and it corresponds to the point on the line  $g(\textbf{U})=0$ with the highest probability. $\nabla g(\textbf{u}^\star)$ is the gradient of $g(\textbf{U})$ evaluated at $\textbf{u}^\star$.

The benchmark problem is a simple structural reliability problem
defined in a two-dimensional parameter space consisting of a resistance $R$ and a stress $S$. The failure happens
when the stress is higher than the resistance, leading to the following limit-state function:

$ \textbf{X}=\{R, S\}$

$g(\textbf{X}) = R - S$

The two random variables are independent  and  distributed
according to:

$R \sim N(200, 20)$

$S \sim N(150, 10)$

In [1]:
#from platform import python_version
#print(python_version())

import numpy as np
from UQpy.Reliability import TaylorSeries
from UQpy.RunModel import RunModel
import matplotlib.pyplot as plt
from UQpy.RunModel import RunModel

In [ ]:
# Monte Carlo simulation

from UQpy.SampleMethods import MCS
N = 100000
x = MCS(dist_name=['normal', 'normal'], dist_params=[[200, 20], [150, 10]], nsamples=N).samples
RunModelObject = RunModel(model_script='pfn.py',model_object_name="example1")
RunModelObject.run(x)
y = np.array(RunModelObject.qoi_list)
idx = sum(i <= 0.0 for i in y)
pf_mcs = idx/N
print(pf_mcs)

In [ ]:
corr = np.array([[1.0, 0.0], [0.0, 1.0]])
RunModelObject = RunModel(model_script='pfn.py',model_object_name="example1") 
Q = TaylorSeries(dimension=2, corr=corr, model=RunModelObject, dist_name=['normal', 'normal'], dist_params=[[200, 20], [150, 10]], n_iter=100, eps=0.0001, df_method='forward')
Q.form()

In [ ]:
# print results
print('Design point in standard normal space: %s' % Q.DesignPoint_U)
print('Design point in original space: %s' % Q.DesignPoint_X)
print('Hasofer-Lind reliability index: %s' % Q.HL_beta)
print('FORM probability of failure: %s' % Q.Prob_FORM)
print('FORM function: %s' % Q.g_record)
print('FORM gradient: %s' % Q.dg_record)
print('FORM x points: %s' % Q.x_record)
print('FORM u points: %s' % Q.u_record)
print('FORM alpha points: %s' % Q.alpha_record)

In [ ]:
# Supporting function
def multivariate_gaussian(pos, mu, Sigma):
    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)
    return np.exp(-fac / 2) / N

N = 60
XX = np.linspace(150, 250, N)
YX = np.linspace(120, 180, N)
XX, YX = np.meshgrid(XX, YX)

XU = np.linspace(-3, 3, N)
YU = np.linspace(-3, 3, N)
XU, YU = np.meshgrid(XU, YU)

# Mean vector and covariance matrix in the original space
parameters = [[200, 20], [150, 10]]
mu_X = np.array([parameters[0][0], parameters[1][0]])
Sigma_X = np.array([[ parameters[0][1]**2 , 0.0], [0.0,  parameters[1][1]**2]])

# Mean vector and covariance matrix in the ostandard normal space
mu_U = np.array([0., 0.])
Sigma_U = np.array([[ 1. , 0.0], [0.0,  1]])

# Pack X and Y into a single 3-dimensional array for the original space
posX = np.empty(XX.shape + (2,))
posX[:, :, 0] = XX
posX[:, :, 1] = YX
ZX = multivariate_gaussian(posX, mu_X, Sigma_X)

# Pack X and Y into a single 3-dimensional array for the standard normal space
posU = np.empty(XU.shape + (2,))
posU[:, :, 0] = XU
posU[:, :, 1] = YU
ZU = multivariate_gaussian(posU, mu_U, Sigma_U)


# Figure 4a
plt.figure()
plt.rcParams["figure.figsize"] = (12, 12)
plt.rcParams.update({'font.size': 22})
plt.plot(parameters[0][0],parameters[1][0],'r.') 
plt.plot([0, 200],[0, 200], 'k', Linewidth=5)
plt.plot(Q.DesignPoint_X[0][0], Q.DesignPoint_X[0][1],'bo',markersize=12)
plt.contour(XX, YX, ZX, levels=20)
plt.xlabel(r'$X_1$')
plt.ylabel(r'$X_2$')
plt.text(170, 182, '$X_1 - X_2=0$',
         rotation=45,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center')
plt.ylim([120, 200])
plt.xlim([130, 240])
plt.grid()
plt.title('Original space')    
plt.axes().set_aspect('equal', 'box')
plt.show()

# Figure 4b
plt.figure()
plt.rcParams["figure.figsize"] = (12, 12)
plt.rcParams.update({'font.size': 22})      
plt.plot([0, Q.DesignPoint_U[0][0]],[0, Q.DesignPoint_U[0][1]], 'b', Linewidth=2)   
plt.plot([0, -3],[5, -1], 'k', Linewidth=5)
plt.plot(Q.DesignPoint_U[0][0], Q.DesignPoint_U[0][1],'bo',markersize=12)
plt.contour(XU, YU, ZU, levels=20)
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.plot(0,0,'r.')

plt.xlabel(r'$U_1$')
plt.ylabel(r'$U_2$')
plt.text(-1.0, 1.1, '$U^\star$=({:1.2f}, {:1.2f})'.format(-2.0, 1.0),
         rotation=0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center')

plt.text(-2.1, 2.05, '$20U_1 - 10U_2 + 50=0$',
         rotation=63,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center')

plt.text(-1.5, 0.7, r'$\overrightarrow{\beta}$',
         rotation=0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center')

plt.text(0.02, -0.2, '({:1.1f}, {:1.1f})'.format(0.0, 0.0))
plt.ylim([-1, 3])
plt.xlim([-3.5, 2])
plt.grid()
plt.title('Standard Normal space')    
plt.axes().set_aspect('equal', 'box')
plt.show()

## Example #2

In the second example we have the limit state to be a linear function of two ($d$=2) independent Gaussian random variables

$g(\textbf{U}) = -\frac{1}{\sqrt{d}}\sum_{i=1}^{d} u_i + \beta$


The probability of failure in this case is $P(F) ≈ 10^{−3}$ for $\beta$ = 3.0902

In [ ]:
RunModelObject2 = RunModel(model_script='pfn.py',model_object_name="example2")
Z = TaylorSeries(dimension=2, model=RunModelObject2, dist_name=['normal', 'normal'], dist_params=[[0, 1], [0, 1]])
Z.form()

In [ ]:
# print results
print('Design point in standard normal space: %s' % Z.DesignPoint_U)
print('Design point in original space: %s' % Z.DesignPoint_X)
print('Hasofer-Lind reliability index: %s' % Z.HL_beta)
print('FORM probability of failure: %s' % Z.Prob_FORM)
print('FORM function: %s' % Z.g_record)
print('FORM gradient: %s' % Z.dg_record)
print('FORM x points: %s' % Z.x_record)
print('FORM u points: %s' % Z.u_record)
print('FORM alpha points: %s' % Z.alpha_record)

In [ ]:
# Figure 4b
plt.figure()
plt.rcParams["figure.figsize"] = (12, 12)
plt.rcParams.update({'font.size': 22}) 
plt.plot([0, Z.DesignPoint_U[0][0]],[0, Z.DesignPoint_U[0][1]], Linewidth=2)                
plt.plot(Z.DesignPoint_U[0][0], Z.DesignPoint_U[0][1],'bo',markersize=12)
plt.contour(XU, YU, ZU, levels=20)
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.plot(0,0,'r.')

plt.xlabel(r'$U_1$')
plt.ylabel(r'$U_2$')
plt.text(2.3, 2.4, '$U^\star$=({:1.2f}, {:1.2f})'.format(2.18, 2.18),
         rotation=0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center')

plt.text(1.0, 0.9, r'$\overrightarrow{\beta}$',
         rotation=0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center')

plt.text(0.02, -0.2, '({:1.1f}, {:1.1f})'.format(0.0, 0.0))
plt.ylim([-1, 3])
plt.xlim([-1, 3])
plt.grid()
plt.title('Standard Normal space')    
plt.axes().set_aspect('equal', 'box')
plt.show()

## Example 3

In the third example we have the limit state to be a linear function of three ($d$=3) independent Gaussian random variables

$g(X_1, X_2, X_3) = 6.2X_1 -X_2X_3^2$

$\mu_{X_1}=20, \mu_{X_2}=5, \mu_{X_3}=4$

$\sigma_{X_1}=3.5, \sigma_{X_2}=0.8, \sigma_{X_3}=0.4$

The probability of failure in this case is $P_f ≈ 0.079$ for $\beta$ = 1.413

In [2]:
corr = np.array([[1.0, 0.0, 0.0],[0.0, 1.0, 0.0], [0.0,0.0, 1.0]])
RunModelObject = RunModel(model_script='pfn.py',model_object_name="example3") 
Q0 = TaylorSeries(dimension=3, corr=corr, model=RunModelObject, dist_name=['normal', 'normal', 'normal'], dist_params=[[20, 3.5], [5, 0.8], [4, 0.4]], n_iter=100, eps=0.0001, df_method='forward')
Q0.form()

Running FORM...
UQpy: Initializing Iterative Translation Approximation Method (ITAM)
UQpy: Computing Nataf correlation distortion...
UQpy: Done.
['UQpy: ITAM iteration number ', 1]
['UQpy: Current error, ', 7.258503754484891e-18]
UQpy: ITAM Done.


/Users/dimitris/PythonProjects/UQpy/src/UQpy/Utilities.py:275: RuntimeWarning: divide by zero encountered in true_divide
  max_ratio = np.amax(np.ones((len(corr), len(corr))) / abs(corr_norm0))


In [3]:
# print results
print('Design point in standard normal space: %s' % Q0.DesignPoint_U)
print('Design point in original space: %s' % Q0.DesignPoint_X)
print('Hasofer-Lind reliability index: %s' % Q0.HL_beta)
print('FORM probability of failure: %s' % Q0.Prob_FORM)
print('FORM function: %s' % Q0.g_record)
print('FORM gradient: %s' % Q0.dg_record)
print('FORM x points: %s' % Q0.x_record)
print('FORM u points: %s' % Q0.u_record)
print('FORM alpha points: %s' % Q0.alpha_record)

Design point in standard normal space: [[-0.94092711  0.65063869  0.82909838]]
Design point in original space: [[16.70675511  5.52051095  4.33163935]]
Hasofer-Lind reliability index: [[1.41282644]]
FORM probability of failure: [[0.07885338]]
FORM function: [44.0, -1.829627253962883, -0.002846818236520221, -1.4417489964557717e-05]
FORM gradient: [array([[ 21.7, -12.8, -16. ]]), array([[ 21.7       , -14.9032874 , -19.01117724]]), array([[ 21.7       , -15.0052639 , -19.12096549]]), array([[ 21.7       , -15.01047958, -19.13028999]])]
FORM x points: [array([[20.,  5.,  4.]]), array([[16.24824582,  5.50583229,  4.31614518]]), array([[16.69525368,  5.51877903,  4.33088673]]), array([[16.70675511,  5.52051095,  4.33163935]])]
FORM u points: [array([0., 0., 0.]), array([[-1.07192977,  0.63229037,  0.79036296]]), array([[-0.94421323,  0.64847379,  0.82721683]]), array([[-0.94092711,  0.65063869,  0.82909838]])]
FORM alpha points: [array([[-0.72708753,  0.42888112,  0.5361014 ]]), array([[-0.6